# Chapter 7: Taproot Dual-Leaf Script Tree> **Reference**: `book/manuscript/Chapter 7 Taproot Dual-Leaf Script Tree - Complete Implementation of Hash Lock and Bob Script.md`  > **Code Examples**: `code/chapter07/`  > **Last Updated**: 2025-12-06---## From Single-Leaf to Dual-Leaf: The True Power of Taproot Script Trees

In the previous chapter, we mastered the complete implementation of single-leaf Taproot Script Path through Alice's Hash Lock contract. However, Taproot's true power lies in its **multi-branch script tree** architecture—the ability to elegantly organize multiple different spending conditions within a single address, implementing complex conditional logic.

Imagine this business scenario: Alice wants to create a digital escrow contract that supports both automatic unlocking based on secret information (Hash Lock) and provides Bob with direct private key control permissions. In traditional Bitcoin, this would require complex multisignature scripts or multiple independent addresses. Taproot's dual-leaf script tree can elegantly integrate these two conditions into a single address:

- **Script Path 1**: Hash Lock script, anyone who knows "helloworld" can spend
- **Script Path 2**: Bob Script, only Bob's private key holder can spend  
- **Key Path**: Alice as the internal key holder can spend directly (maximum privacy)

The elegance of this design lies in the fact that external observers cannot distinguish whether this is a simple payment or a complex three-path conditional contract. Only during actual spending is the used path selectively revealed.

Unlike single-leaf trees that directly use TapLeaf hash as the Merkle root, dual-leaf script trees require building a true Merkle tree:```        Merkle Root       /           \  TapLeaf A    TapLeaf B(Hash Script) (Bob Script)```**Technical Implementation Key Points**:1. **TapLeaf Hash Calculation**: Each script separately calculates its TapLeaf hash2. **TapBranch Hash Calculation**: Calculate TapBranch hash after sorting the two TapLeaf hashes lexicographically  3. **Control Block Construction**: Each script needs to include its sibling node hash as Merkle proofLet's deeply understand how all this works through actual on-chain transaction data.

We will analyze the complete implementation of dual-leaf script trees based on two real testnet transactions:### Transaction 1: Hash Script Path Spending- **Transaction ID**: `b61857a05852482c9d5ffbb8159fc2ba1efa3dd16fe4595f121fc35878a2e430`- **Taproot Address**: `tb1p93c4wxsr87p88jau7vru83zpk6xl0shf5ynmutd9x0gxwau3tngq9a4w3z`- **Spending Method**: Script Path (using preimage "helloworld")### Transaction 2: Bob Script Path Spending- **Transaction ID**: `185024daff64cea4c82f129aa9a8e97b4622899961452d1d144604e65a70cfe0`- **Taproot Address**: `tb1p93c4wxsr87p88jau7vru83zpk6xl0shf5ynmutd9x0gxwau3tngq9a4w3z`- **Spending Method**: Script Path (using Bob's private key signature)Note that these two transactions use the **exact same Taproot address**, proving they indeed originate from the same dual-leaf script tree!

First, let's reconstruct the complete code that generates this Taproot address:**Key Technical Details**:1. **Flat Structure**: `all_leafs = [hash_script, bob_script]` indicates two scripts at the same level2. **Index Order**: hash_script is index 0, bob_script is index 13. **Address Consistency**: Two different Script Path spends using the same address proves correct script tree construction

After mastering the dual-leaf script tree construction principles, let's see how to implement two different Script Path spends in the Reveal phase.### Hash Script Path Spending Core CodeBased on transaction `b61857a05852482c9d5ffbb8159fc2ba1efa3dd16fe4595f121fc35878a2e430` implementation:### Bob Script Path Spending Core CodeBased on transaction `185024daff64cea4c82f129aa9a8e97b4622899961452d1d144604e65a70cfe0` implementation:**Key Technical Comparison**:| Aspect | Hash Script Path | Bob Script Path ||--------|------------------|-----------------|| **Script Index** | 0 (first script) | 1 (second script) || **Input Data** | preimage hex | Schnorr signature || **Verification Method** | Hash matching | Digital signature verification || **Control Block** | Contains Bob Script's TapLeaf hash | Contains Hash Script's TapLeaf hash |

In dual-leaf script trees, each script's Control Block contains its sibling node hash as Merkle proof. Let's analyze the actual on-chain data:### Hash Script Path Control Block**Data extracted from transaction b61857a0...**:```Control Block: c050be5fc44ec580c387bf45df275aaa8b27e2d7716af31f10eeed357d126bb4d32faaa677cb6ad6a74bf7025e4cd03d2a82c7fb8e3c277916d7751078105cf9dfStructure breakdown:├─ c0: Leaf version (0xc0)├─ 50be5fc44ec580c387bf45df275aaa8b27e2d7716af31f10eeed357d126bb4d3: Alice internal pubkey└─ 2faaa677cb6ad6a74bf7025e4cd03d2a82c7fb8e3c277916d7751078105cf9df: Bob Script's TapLeaf hash```### Bob Script Path Control Block**Data extracted from transaction 185024da...**:```Control Block: c050be5fc44ec580c387bf45df275aaa8b27e2d7716af31f10eeed357d126bb4d3fe78d8523ce9603014b28739a51ef826f791aa17511e617af6dc96a8f10f659eStructure breakdown:├─ c0: Leaf version (0xc0)├─ 50be5fc44ec580c387bf45df275aaa8b27e2d7716af31f10eeed357d126bb4d3: Alice internal pubkey (same!)└─ fe78d8523ce9603014b28739a51ef826f791aa17511e617af6dc96a8f10f659e: Hash Script's TapLeaf hash```**Important Observations**:- Both Control Blocks use the **same internal public key**- Merkle Path portions are **sibling node** TapLeaf hashes- This is exactly the manifestation of Merkle tree structure!### Control Block Verification AlgorithmVerifying Control Block essentially means **address reconstruction verification**:

Now let's analyze the complete execution process of Hash Script Path in detail. Based on actual data from transaction `b61857a0...`:### Witness Data Structure```Witness Stack:[0] 68656c6c6f776f726c64                                                     (preimage_hex)[1] a820936a185caaa266bb9cbe981e9e05cb78cd732b0b3280eb944412bb6f8f8f07af8851   (script_hex)[2] c050be5fc44ec580c387bf45df275aaa8b27e2d7716af31f10eeed357d126bb4d32faaa677cb6ad6a74bf7025e4cd03d2a82c7fb8e3c277916d7751078105cf9df (control_block)```### Script Bytecode Parsing**Hash Script**: `a820936a185caaa266bb9cbe981e9e05cb78cd732b0b3280eb944412bb6f8f8f07af8851````Bytecode breakdown:a8 = OP_SHA25620 = OP_PUSHBYTES_32936a185caaa266bb9cbe981e9e05cb78cd732b0b3280eb944412bb6f8f8f07af = SHA256("helloworld")88 = OP_EQUALVERIFY51 = OP_PUSHNUM_1 (OP_TRUE)```### Stack Execution Animation - Hash Script Path**Executing script**: `OP_SHA256 OP_PUSHBYTES_32 936a185caaa266bb9cbe981e9e05cb78cd732b0b3280eb944412bb6f8f8f07af OP_EQUALVERIFY OP_PUSHNUM_1`#### 0. Initial state: Load script input```| 68656c6c6f776f726c64 (preimage_hex) |└──────────────────────────────────────┘```**(Preimage "helloworld" hex representation already on stack)**#### 1. OP_SHA256: Calculate SHA256 hash of top stack element```| 936a185caaa266bb9cbe981e9e05cb78cd732b0b3280eb944412bb6f8f8f07af (computed_hash) |└─────────────────────────────────────────────────────────────────────────────────────┘```**(SHA256("helloworld") = 936a185c...07af)**#### 2. OP_PUSHBYTES_32: Push expected hash value```| 936a185caaa266bb9cbe981e9e05cb78cd732b0b3280eb944412bb6f8f8f07af (expected_hash) || 936a185caaa266bb9cbe981e9e05cb78cd732b0b3280eb944412bb6f8f8f07af (computed_hash) |└─────────────────────────────────────────────────────────────────────────────────────┘```**(Two identical hash values now at stack top)**#### 3. OP_EQUALVERIFY: Verify hash equality```| (empty_stack) |└───────────────┘```**(Verification successful: expected_hash == computed_hash, both elements removed)**#### 4. OP_PUSHNUM_1: Push success flag```| 01 (true_value) |└─────────────────┘```**(Script execution successful: non-zero value at stack top)**

Next, let's analyze Bob Script Path execution process. Based on actual data from transaction `185024da...`:### Witness Data Structure```Witness Stack:[0] 26a0eadca0bba3d1bb6f82b8e1f76e2d84038c97a92fa95cc0b9f6a6a59bac5f9977d7cb33dbd188b1b84e6d5a9447231353590578f358b2f18a66731f9f1c5c (bob_signature)[1] 2084b5951609b76619a1ce7f48977b4312ebe226987166ef044bfb374ceef63af5ac                                                               (script_hex)[2] c050be5fc44ec580c387bf45df275aaa8b27e2d7716af31f10eeed357d126bb4d3fe78d8523ce9603014b28739a51ef826f791aa17511e617af6dc96a8f10f659e   (control_block)```### Script Bytecode Parsing**Bob Script**: `2084b5951609b76619a1ce7f48977b4312ebe226987166ef044bfb374ceef63af5ac````Bytecode breakdown:20 = OP_PUSHBYTES_3284b5951609b76619a1ce7f48977b4312ebe226987166ef044bfb374ceef63af5 = Bob's x-only pubkeyac = OP_CHECKSIG```### Stack Execution Animation - Bob Script Path**Executing script**: `OP_PUSHBYTES_32 84b5951609b76619a1ce7f48977b4312ebe226987166ef044bfb374ceef63af5 OP_CHECKSIG`#### 0. Initial state: Load script input```| 26a0eadca0bba3d1bb6f82b8e1f76e2d84038c97a92fa95cc0b9f6a6a59bac5f9977d7cb33dbd188b1b84e6d5a9447231353590578f358b2f18a66731f9f1c5c (bob_signature) |└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘```**(Bob's 64-byte Schnorr signature already on stack)**#### 1. OP_PUSHBYTES_32: Push Bob's x-only pubkey```| 84b5951609b76619a1ce7f48977b4312ebe226987166ef044bfb374ceef63af5 (bob_pubkey)   || 26a0eadca0bba3d1bb6f82b8e1f76e2d84038c97a92fa95cc0b9f6a6a59bac5f9977d7cb33dbd188b1b84e6d5a9447231353590578f358b2f18a66731f9f1c5c (bob_signature) |└───────────────────────────────────────────────────────────────────────────────────┘```**(Bob's 32-byte x-only pubkey pushed to stack top)**#### 2. OP_CHECKSIG: Verify Schnorr signature```| 01 (signature_valid) |└──────────────────────┘```**(Signature verification successful: Bob's private key corresponds to this pubkey, and signature is valid for transaction data)****Verification Process Details**:1. Pop pubkey from stack: `84b5951609b76619a1ce7f48977b4312ebe226987166ef044bfb374ceef63af5`2. Pop signature from stack: `26a0eadca0bba3d1bb6f82b8e1f76e2d84038c97a92fa95cc0b9f6a6a59bac5f...`3. Use BIP340 Schnorr signature verification algorithm to verify signature validity4. Verification successful, push 1 to indicate TRUE

By comparing single-leaf and dual-leaf implementations, we can clearly see the differences in Merkle tree calculations:### Single-Leaf Script Tree```Merkle Root = TapLeaf Hash            = Tagged_Hash("TapLeaf", 0xc0 + len(script) + script)```**Characteristics**:- Simple and direct, TapLeaf hash serves as Merkle root- Control Block only contains internal pubkey, no Merkle path- Suitable for simple single-condition verification scenarios### Dual-Leaf Script Tree```Merkle Root = TapBranch Hash            = Tagged_Hash("TapBranch", sorted(TapLeaf_A, TapLeaf_B))TapLeaf_A = Tagged_Hash("TapLeaf", 0xc0 + len(script_A) + script_A)TapLeaf_B = Tagged_Hash("TapLeaf", 0xc0 + len(script_B) + script_B)```**Characteristics**:- True Merkle tree structure, requires TapBranch calculation- Lexicographic ordering ensures deterministic results- Control Block contains sibling node hash as Merkle proof- Supports complex multi-condition verification scenarios### Control Block Size Comparison| Script Tree Type | Control Block Size | Structure ||------------------|-------------------|-----------|| Single-leaf | 33 bytes | [version+parity] + [internal_pubkey] || Dual-leaf | 65 bytes | [version+parity] + [internal_pubkey] + [sibling_hash] || Four-leaf | 97 bytes | [version+parity] + [internal_pubkey] + [sibling_hash] + [parent_sibling_hash] |As script tree depth increases, Control Block grows linearly, but is still much more efficient than traditional multisignature scripts.

Based on the previous analysis, let's summarize development best practices for dual-leaf Taproot applications:### 1. Standard Commit Phase Workflow### 2. Universal Script Path Spending Template### 3. Common Errors and Debugging Tips**Script Index Errors**:**Merkle Path Verification Failure**:

Through actual on-chain data, we can quantitatively analyze performance and privacy characteristics of different spending methods:| Spending Method | Transaction Size | Witness Data | Computational Complexity | Privacy Level | Relative Fee Cost ||-----------------|------------------|--------------|-------------------------|---------------|-------------------|| **Key Path** | ~110 bytes | 64-byte signature | 1 signature verification | Complete privacy | Baseline (1.0x) || **Hash Script** | ~180 bytes | preimage+script+cb | Hash calculation+Merkle verification | Exposes Hash Lock | Medium (1.6x) || **Bob Script** | ~185 bytes | signature+script+cb | Signature verification+Merkle verification | Exposes P2PK structure | Medium (1.7x) |**Key Insights**:1. **Key Path is always the optimal choice**: Regardless of script tree complexity, Key Path has the highest efficiency and privacy2. **Script Path costs are manageable**: Compared to traditional complex scripts, Taproot's additional overhead is within acceptable range3. **Value of selective revelation**: Only the actually used path is exposed, unused paths remain private forever

Through the complete implementation of dual-leaf script trees, we have mastered the key technologies of Taproot multi-path spending: true Merkle tree construction, Control Blocks containing sibling node proofs, and coordination mechanisms for different scripts within the same address. More importantly, we understand Taproot's core philosophy—selective revelation, where only the used path is exposed, achieving a perfect balance between complex functionality and high privacy.In the next chapter, we will explore **multi-layer nested script trees** and **advanced Taproot application patterns**, learning how to build enterprise-grade blockchain applications supporting more spending conditions, and how to combine time locks, multisignature, and other advanced features to create more complex and practical smart contract systems.Dual-leaf script trees are an important milestone in Taproot application development—they demonstrate how to achieve true functional complexity while maintaining simplicity. This is the essence of Bitcoin Taproot technology: **Simple in appearance, powerful within**.